In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

## archives.gov electoral vote scraper

In [ ]:
def alfa(x): return ''.join([c for c in x if c.isalpha() or c==' '])
def wl(x):
    f = open('latek-output.tex', 'wt')
    f.write(x.to_latex())
    f.close()

In [ ]:
elec_vote_map = dict()

if False:
    for y in range(1976, 2021, 4):
        uri = f'https://www.archives.gov/electoral-college/{y}'
        r = requests.get(uri)
        states = bs(r.text, 'html.parser').findAll('tbody')[1].findAll('tr')
        for i in range(2,53):
            cell_list = states[i].findAll('td')

            s = alfa(cell_list[0].text).upper()
            ev = int(cell_list[1].text)
            elec_vote_map[(y,s)] = ev
            pickle.dump(elec_vote_map, open('datasets/elec_vote_map.pkl', 'wb'))
else:
    elec_vote_map = pickle.load(open('datasets/elec_vote_map.pkl', 'rb'))

In [ ]:
df = pd.read_csv('datasets-clean/xxx-fitted-values.csv')
df = df[['year', 'state', 'y_votes_percent', 'y_hat_fr', 'y_hat_ds', ]]

df['y_err_fr'] = abs(df.y_votes_percent-df.y_hat_fr)
df['y_err_ds'] = abs(df.y_votes_percent-df.y_hat_ds)

for i in df.index:
    y = df.at[i, 'year']
    s = df.at[i, 'state']

    df.at[i, 'y_ev'] = elec_vote_map[(y,s)] if df.at[i, 'y_votes_percent'] > 50 else 0

    df.at[i, 'y_fr'] = elec_vote_map[(y,s)] if df.at[i, 'y_hat_fr'] > 50 else 0
    df.at[i, 'y_ds'] = elec_vote_map[(y,s)] if df.at[i, 'y_hat_ds'] > 50 else 0

    df.at[i, 'fr_spill'] = df.at[i, 'y_fr'] != df.at[i, 'y_ev']
    df.at[i, 'ds_spill'] = df.at[i, 'y_ds'] != df.at[i, 'y_ev']

In [ ]:
sum(df.ds_spill) / (10*50)

In [ ]:
wl(df[df.year == 2012][df.fr_spill][['state', 'y_votes_percent', 'y_hat_fr', ]])

In [ ]:
# solo sulla percentuale, non sulla assegnazione
np.sqrt(mean_squared_error(df.y_votes_percent, df.y_hat_fr)), np.sqrt(mean_squared_error(df.y_votes_percent, df.y_hat_ds))

In [ ]:
df.to_csv('XXX-final-final-prediction-fitted-values.csv',)

In [ ]:
(np.mean(df.y_err_fr) /  np.mean(df.y_err_ds) - 1)*100 , 100 - (np.mean(df.y_err_ds) / np.mean(df.y_err_fr))*100

In [ ]:
tab = df.groupby('year')['y_ev', 'y_fr', 'y_ds',].sum()
tab.y_ev += 3
tab.y_fr += 3
tab.y_ds += 3
tab.y_ev = tab.y_ev.astype(int)
tab.y_fr = tab.y_fr.astype(int)
tab.y_ds = tab.y_ds.astype(int)
tab